<a href="https://colab.research.google.com/github/YangTaeSung/CAU-MachineLearning/blob/master/assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
'''from google.colab import drive
drive.mount('content/drive')'''

"from google.colab import drive\ndrive.mount('content/drive')"

In [112]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


review_data = load_files(r"/content/drive/My Drive/movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
print(X_train)
print(y_train)
print(X_test)
print(y_test)
print(len(y_test))
print(len(y_train))

[[0.         0.         0.         ... 0.05798721 0.01984438 0.        ]
 [0.         0.06868034 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[1 0 1 ... 1 0 1]
[[0.         0.         0.         ... 0.         0.08380802 0.07280323]
 [0.         0.         0.09645829 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04428295 0.         0.         ... 0.07607835 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[1 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0
 1 0 1 1 0 1 1 1 1 

In [114]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0


In [115]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(610,)

In [0]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='sigmoid'),
    keras.layers.Dense(10, activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [118]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
45/45 [==============================] - 0s 3ms/step - loss: 1.3366 - accuracy: 0.5063
Epoch 2/10
45/45 [==============================] - 0s 3ms/step - loss: 0.7691 - accuracy: 0.5063
Epoch 3/10
45/45 [==============================] - 0s 3ms/step - loss: 0.7181 - accuracy: 0.5063
Epoch 4/10
45/45 [==============================] - 0s 2ms/step - loss: 0.7066 - accuracy: 0.5063
Epoch 5/10
45/45 [==============================] - 0s 2ms/step - loss: 0.7014 - accuracy: 0.5063
Epoch 6/10
45/45 [==============================] - 0s 2ms/step - loss: 0.6982 - accuracy: 0.5063
Epoch 7/10
45/45 [==============================] - 0s 2ms/step - loss: 0.6956 - accuracy: 0.5063
Epoch 8/10
45/45 [==============================] - 0s 3ms/step - loss: 0.6917 - accuracy: 0.5063
Epoch 9/10
45/45 [==============================] - 0s 3ms/step - loss: 0.6836 - accuracy: 0.5063
Epoch 10/10
45/45 [==============================] - 0s 3ms/step - loss: 0.6626 - accuracy: 0.5091


In [119]:
test_loss, test_acc = model.evaluate(X_train,  y_train, verbose=2)

print('\n테스트 정확도:', test_acc)

45/45 - 0s - loss: 0.6107 - accuracy: 0.7447

테스트 정확도: 0.74472576379776


In [120]:
predictions = model.predict(X_test)
np.argmax(predictions[0])

0

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')